# POC: Integração de Simulações Cooja com MongoDB

## Objetivo

Este notebook implementa uma **Prova de Conceito (POC)** para orquestrar simulações de redes IoT no **Cooja (Contiki-NG)** usando **MongoDB** como sistema de controle e armazenamento de metadados. O sistema está preparado para funcionar em um ambiente conteinerizado com Docker.

Este fluxo de execuação inclui:
- Organização de dados com MongoDB.
- Geração de redes aleatórias.
- Transformação e conversão de arquivos de configurações do Cooja.
- Execução de simulações e registros dos logs de execução.

## Requisitos

Certifique-se de que os seguintes componentes estejam instalados:

- Python **3.12** ou superior
- Biblioteca `pymongo`
- Docker e Docker Compose
- Imagem Docker [ssh-cooja-docker](https://github.com/JunioCesarFerreira/Cooja-Docker-VM-Setup/tree/main/ssh-docker-cooja)
- Replica set do MongoDB ativado (`rs0`)
- Execução do Docker Compose localizado em `simlab/docker-compose.yaml`

## Organização do Notebook

### 1. Configurações Iniciais
```python
MONGO_URI = "mongodb://localhost:27017/?replicaSet=rs0"
DB_NAME = "simlab"
```
Define a URI para conexão com a instância do MongoDB e o nome do banco de dados utilizado.

### 2. Classe de Interface com o MongoDB
A classe responsável por encapsular a lógica de leitura, escrita e atualização de documentos no MongoDB é implementada neste trecho. Ela fornece métodos como:
- Obter experimentos pendentes
- Gerar populações
- Atualizar status de simulações
- Registrar resultados

Essa estrutura permite o controle da fila de simulações, gerenciando ciclos de evolução.

### 3. Lógica de População e Geração
Funções responsáveis por:
- Capturar experimentos na fila (`get_first_waiting_experiment`)
- Gerar posições aleatórias para os motes
- Gerar arquivos `.xml` e `.dat` que são usados como entrada pelo Cooja

### 4. Execução das Simulações
Contém a lógica para orquestrar:
- Geração da simulação
- Execução via SSH ou Docker remoto
- Monitoramento da execução
- Registro dos resultados de volta no MongoDB


### 5. Exemplos de Execução
Contém células que mostram como testar manualmente os métodos da classe, populando a fila, iniciando uma geração, e registrando os resultados de forma simulada.

## Estrutura dos Diretórios

- `data`: Dados de entrada para execução de simulações. Neste diretório esta o arquivo `json` principal que contém as configurações de experimento.
- `logs`: Diretório onde são registrados os logs de execução do Cooja.
- `simlab`: Diretório base do Docker-Compose desta POC.
- `tmp`: Diretório de uso temporário para transferências de arquivos.

## Utilitários

Neste mesmo diretório, você também encontrará:

- `util-clear-tmp.py`: Programa para limpeza dos logs e arquivos temporários.
- `util-plot-pos.py`: Programa para plotagem de pontos gerados para simulações.
- `util-send-vm.py`: Programa para envio de arquivos de configurações para execução no Cooja em VM.


## MongoDB Methods

Configurações do banco de dados MongoDB

In [1]:
MONGO_URI = "mongodb://localhost:27017/?replicaSet=rs0"
DB_NAME = "simlab"

Classe que concentra os métodos de manipulação do MongoDB para esta aplicação.

In [2]:
from pymongo import MongoClient
import gridfs
from bson import ObjectId
from datetime import datetime
from typing import Optional
import time

class MongoExperimentManager:
    def __init__(self, mongo_uri: str, db_name: str, info_log: bool = False):
        self.mongo_uri = mongo_uri
        self.db_name = db_name
        self.info_log = info_log

    def _get_client(self):
        return MongoClient(self.mongo_uri)
        
#---------------------
# Métodos Genéricos
#---------------------
    # Armazena um arquivo no GridFS e retorna o ID.
    def insert_file(self, path: str, name: str) -> str:
        client = self._get_client()
        db = client[self.db_name]
        fs = gridfs.GridFS(db)

        with open(path, "rb") as f:
            file_id = fs.put(f, filename=name)

        client.close()
        return ObjectId(file_id)

    # Recupera um arquivo do GridFS e salva ele no local indicado
    def save_file_from_mongo(self, file_id, local_path):
        try:
            client = self._get_client()
            db = client[self.db_name]
            fs = gridfs.GridFS(db)

            grid_out = fs.get(ObjectId(file_id))

            with open(local_path, 'wb') as f:
                f.write(grid_out.read())
                
            if self.info_log:
                print(f"[INFO] Arquivo {local_path} salvo com sucesso.")
                
        except Exception as e:
            print(f"[ERRO] Falha ao salvar arquivo {file_id}: {e}")
            
        finally:
            client.close()

    # Insere um novo documento
    def insert_document(self, document: dict, collection_name: str) -> str:
        client = self._get_client()
        db = client[self.db_name]
        collection = db[collection_name]

        result = collection.insert_one(document)
        client.close()

        return ObjectId(result.inserted_id)

    # Pega um documento por ID
    def get_document_by_id(self, collection_name: str, document_id: ObjectId) -> Optional[dict]:
        client = self._get_client()
        db = client[self.db_name]
        collection = db[collection_name]

        result = collection.find_one({"_id": document_id})

        client.close()
        return result
    
    def get_collection(self, coll_name):
        client = self._get_client()
        db = client[self.db_name]
        return db[coll_name]
   
#---------------------
# Métodos Específicos
#--------------------- 
    # Insere um novo experimento
    def insert_experiment(
        self,
        experiment_data: dict,
        file_parameters: Optional[list[dict]] = None
    ) -> str:
        """
        Insere um experimento no MongoDB conforme a estrutura do modelo Experiment.
        """
        client = self._get_client()
        db = client[self.db_name]
        collection = db["experiments"]

        document = {
            "name": experiment_data.get("name", ""),
            "status": experiment_data.get("status", "Waiting"),
            "enqueuedTime": datetime.now(),
            "evolutiveParameters": experiment_data.get("evolutiveParameters", {}),
            "simulationModel": experiment_data.get("simulationModel", {}),
            "linkedFiles": [],
            "generations": []
        }

        if file_parameters:
            for file_param in file_parameters:
                try:
                    file_id = self.insert_file(file_param["filePath"], file_param["name"])
                    linked_file = {
                        "name": file_param["name"],
                        "fileId": file_id
                    }
                    document["linkedFiles"].append(linked_file)
                except Exception as e:
                    print(f"Erro ao processar arquivo {file_param['filePath']}: {str(e)}")
                    continue

        result = collection.insert_one(document)
        client.close()

        return ObjectId(result.inserted_id)

    # Retorna experimentos que estão com status Waiting
    def get_waiting_experiments(self) -> list[dict]:
        """
        Recupera todos os experimentos com status 'Waiting'.
        """
        client = self._get_client()
        db = client[self.db_name]
        collection = db["experiments"]

        waiting_experiments = list(collection.find({"status": "Waiting"}))

        client.close()
        return waiting_experiments

    # Retorna o primeiro experimento que esteja com status Waiting
    def get_first_waiting_experiment(self) -> Optional[dict]:
        """
        Recupera o primeiro experimento com status 'Waiting'.
        """
        client = self._get_client()
        db = client[self.db_name]
        collection = db["experiments"]

        experiment = collection.find_one({"status": "Waiting"})

        client.close()
        return experiment

    def find_pending_simulations(self):
        client = self._get_client()
        db = client[self.db_name]
        simqueue_coll = db["simqueue"]
        return simqueue_coll.find({"status": "waiting"})

    def update_experiment(self, experiment_id: str, updates: dict) -> bool:
        """
        Atualiza os campos de um experimento existente no MongoDB.
        """
        client = self._get_client()
        db = client[self.db_name]
        collection = db["experiments"]

        result = collection.update_one(
            {"_id": ObjectId(experiment_id)},
            {"$set": updates}
        )

        client.close()
        return result.modified_count > 0
            
    def update_simulation_status(self, sim_id, new_status):
        client = self._get_client()
        db = client[self.db_name]
        simqueue_coll = db["simqueue"]
        simqueue_coll.update_one(
            {"_id": ObjectId(sim_id)},
            {"$set": {"status": new_status, "timestamp": time.time()}}
        )
        print(f"[MongoDB] Simulação {sim_id} atualizada para status: {new_status}")
        
    def simulation_done(self, sim: dict, log_result_id: str):        
        # Atualiza o documento da simulação na coleção "simqueue":
        # define o status "done" e o campo simLogFile com o ID do log.
        client = self._get_client()
        db = client[self.db_name]
        simqueue_coll = db["simqueue"]
        generations_coll = db["generations"]
            
        log_oid = ObjectId(log_result_id)
    
        simqueue_coll.update_one(
            { "_id": sim["_id"] },
            { "$set": { "simLogFile": log_result_id, "status": "done", "timestamp": time.time() }}
        )
        
        update_result = generations_coll.update_one(
            { "_id": sim["generation_id"] },
            {
                "$set": {
                    "population.$[ind].simLogFile": log_oid
                }
            },
            array_filters=[
                { "ind.simulationFile": sim["simulationFile"] }
            ]
        )
        if update_result.matched_count == 0:
            print("[WARN] Nenhum indivíduo correspondente encontrado na geração.")
        
        client.close()

# Factory usando atributos globais
def mongo_exp_mgr_factory()-> MongoExperimentManager:
    return MongoExperimentManager(MONGO_URI, DB_NAME)

O diretório `data`contém os dados de entrada para execução de um experimento. Incluindo:

- `inputExample.json`: Arquivo principal de configurações dos experimentos.

- `outros códigos`: Arquivos de códigos adicionais para realização das simulações.

O trecho de código a seguir realiza abertura destes arquivos e registra um experimento no MongoDB.

In [3]:
import json
from pathlib import Path

DATA_DIR = Path("data")
INPUT_FILE = DATA_DIR / "inputExample.json"

def new_experiment():
    # Carrega os dados do arquivo JSON
    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        experiment_data = json.load(f)

    # Extrai a lista de arquivos
    linked_files = experiment_data.get("linkedFiles", [])
    file_parameters = []

    # Prepara lista de arquivos vinculados
    for f in linked_files:
        file_path = DATA_DIR / f["name"]
        if file_path.exists():
            file_parameters.append({
                "name": f["name"],
                "filePath": str(file_path)
            })
        else:
            print(f"Aviso: Arquivo não encontrado: {file_path}")

    mdb = mongo_exp_mgr_factory()
    inserted_id = mdb.insert_experiment(experiment_data, file_parameters)
    print(f"Experimento inserido com ID: {inserted_id}")

new_experiment()

Experimento inserido com ID: 68052f264b7b23d6ec07a558


## Util Methods

### network_random_point

Função para gerar pontos aleatórios maximizando a cobertura da região, garantindo que o grafo seja conexo. Isto é, este método gera uma rede aleatória, porém funcional.

In [4]:
import random
import math
import numpy as np
from collections import deque

def network_random_points(amount, region, radius, max_attempts=50):
    """
    Gera pontos aleatórios maximizando a cobertura da região, garantindo que o grafo seja conexo.
    
    Parâmetros:
    - amount: número de pontos a serem gerados
    - region: tuple (x_min, y_min, x_max, y_max) definindo a região retangular
    - radius: raio de conexão entre pontos
    - max_attempts: tentativas máximas para encontrar posição válida para cada ponto
    
    Retorna:
    - Lista de tuples (x, y) com as coordenadas dos pontos
    """
    
    if amount <= 0:
        return []
    
    x_min, y_min, x_max, y_max = region
    points = []
    
    # 1. Primeiro ponto no centro da região
    first_x = (x_min + x_max) / 2
    first_y = (y_min + y_max) / 2
    points.append((first_x, first_y))
    
    # Função para verificar se o grafo é conexo
    def is_connected(points, radius):
        if not points:
            return True
        visited = set()
        queue = deque([0])
        adjacency = {i: [] for i in range(len(points))}
        
        # Construir lista de adjacência
        for i in range(len(points)):
            for j in range(i+1, len(points)):
                if math.dist(points[i], points[j]) <= radius:
                    adjacency[i].append(j)
                    adjacency[j].append(i)
        
        # BFS para verificar conectividade
        while queue:
            node = queue.popleft()
            if node not in visited:
                visited.add(node)
                queue.extend(adjacency[node])
        
        return len(visited) == len(points)
    
    # 2. Geração de pontos com garantia de conectividade
    while len(points) < amount:
        best_point = None
        max_min_distance = 0
        
        for _ in range(max_attempts):
            # Escolhe um ponto âncora aleatório entre os pontos conectados
            connected_indices = list(range(len(points)))  # Todos inicialmente conectados
            anchor_idx = random.choice(connected_indices)
            anchor = points[anchor_idx]
            
            angle = random.uniform(0, 2 * math.pi)
            distance = random.uniform(0.5 * radius, radius)  # Intervalo mais conservador
            
            new_x = anchor[0] + distance * math.cos(angle)
            new_y = anchor[1] + distance * math.sin(angle)
            
            # Verifica se está dentro da região
            if not (x_min <= new_x <= x_max and y_min <= new_y <= y_max):
                continue
                
            # Ponto candidato temporário
            temp_points = points + [(new_x, new_y)]
            
            # Verifica conectividade
            if is_connected(temp_points, radius):
                # Calcula distância mínima para maximizar cobertura
                min_dist = min(math.dist((new_x, new_y), p) for p in points)
                if min_dist > max_min_distance:
                    max_min_distance = min_dist
                    best_point = (new_x, new_y)
        
        # Adiciona o melhor ponto encontrado ou usa estratégia alternativa
        if best_point:
            points.append(best_point)
        else:
            # Estratégia alternativa: conecta ao ponto mais próximo desconectado
            if len(points) > 1:
                # Encontra componentes desconexos
                components = []
                visited = set()
                for i in range(len(points)):
                    if i not in visited:
                        component = []
                        queue = deque([i])
                        while queue:
                            node = queue.popleft()
                            if node not in visited:
                                visited.add(node)
                                component.append(node)
                                for j in range(len(points)):
                                    if j != node and math.dist(points[node], points[j]) <= radius:
                                        queue.append(j)
                        components.append(component)
                
                # Se houver múltiplos componentes, conecta-os
                if len(components) > 1:
                    # Escolhe dois componentes aleatórios
                    comp1, comp2 = random.sample(components, 2)
                    point1 = points[random.choice(comp1)]
                    point2 = points[random.choice(comp2)]
                    
                    # Gera ponto intermediário que conecta ambos
                    mid_x = (point1[0] + point2[0]) / 2
                    mid_y = (point1[1] + point2[1]) / 2
                    direction = math.atan2(point2[1]-point1[1], point2[0]-point1[0])
                    
                    # Ajusta para ficar dentro do raio de ambos
                    new_x = mid_x + random.uniform(-0.3*radius, 0.3*radius) * math.sin(direction)
                    new_y = mid_y + random.uniform(-0.3*radius, 0.3*radius) * math.cos(direction)
                    
                    # Garante que está dentro da região
                    new_x = np.clip(new_x, x_min, x_max)
                    new_y = np.clip(new_y, y_min, y_max)
                    
                    points.append((new_x, new_y))
                    continue
            
            # Fallback: ponto aleatório conectado ao componente principal
            anchor = random.choice(points)
            angle = random.uniform(0, 2 * math.pi)
            distance = random.uniform(0, radius)
            new_x = anchor[0] + distance * math.cos(angle)
            new_y = anchor[1] + distance * math.sin(angle)
            
            new_x = np.clip(new_x, x_min, x_max)
            new_y = np.clip(new_y, y_min, y_max)
            points.append((new_x, new_y))
    
    # 3. Verificação final de conectividade
    if not is_connected(points, radius):
        # Força conexão se necessário
        components = []
        visited = set()
        for i in range(len(points)):
            if i not in visited:
                component = []
                queue = deque([i])
                while queue:
                    node = queue.popleft()
                    if node not in visited:
                        visited.add(node)
                        component.append(node)
                        for j in range(len(points)):
                            if j != node and math.dist(points[node], points[j]) <= radius:
                                queue.append(j)
                components.append(component)
        
        while len(components) > 1:
            # Conecta dois componentes aleatórios
            comp1, comp2 = random.sample(components, 2)
            point1 = points[random.choice(comp1)]
            point2 = points[random.choice(comp2)]
            
            # Adiciona ponto intermediário
            mid_x = (point1[0] + point2[0]) / 2
            mid_y = (point1[1] + point2[1]) / 2
            points.append((mid_x, mid_y))
            
            # Recalcula componentes
            components = []
            visited = set()
            for i in range(len(points)):
                if i not in visited:
                    component = []
                    queue = deque([i])
                    while queue:
                        node = queue.popleft()
                        if node not in visited:
                            visited.add(node)
                            component.append(node)
                            for j in range(len(points)):
                                if j != node and math.dist(points[node], points[j]) <= radius:
                                    queue.append(j)
                    components.append(component)
    
    return points

### generate_positions_from_json

No bloco de código a seguir, definimos a estrutura elementar de elementos de simulação, além disso, definimos a função que realiza conversão desta estrtura em arquivo *positions.dat* do Cooja.

In [5]:
import numpy as np
from typing import TypedDict

# Redefinindo tipos para type checking
class FixedMote(TypedDict):
    position: list[float]
    name: str
    sourceCode: str

class MobileMote(TypedDict):
    functionPath: list[tuple[str, str]]  # Agora é uma lista de tuplas
    isClosed: bool
    isRoundTrip: bool
    speed: float
    timeStep: float
    name: str
    sourceCode: str

class SimulationElements(TypedDict):
    fixedMotes: list[FixedMote]
    mobileMotes: list[MobileMote]

class SimulationConfig(TypedDict):
    name: str
    duration: float
    radiusOfReach: float
    region: tuple[float, float, float, float]
    simulationElements: SimulationElements

def evaluate_function(expression: str, t_values: np.ndarray) -> np.ndarray:
    return np.array([eval(expression, {"t": t, "np": np}) for t in t_values])

def generate_positions_from_json(
    config: SimulationConfig, 
    output_filename: str = "positions.dat"
    ) -> tuple[list[tuple[float, float]], list[tuple[float, float]]]:

    fixed_positions = [(mote["position"][0], mote["position"][1]) for mote in config["simulationElements"]["fixedMotes"]]
    mobile_motes = config["simulationElements"]["mobileMotes"]
    
    mobile_start_positions = []

    with open(output_filename, "w") as file:
        file.write("# Fixed positions\n")
        for i, (x, y) in enumerate(fixed_positions):
            file.write(f"{i} 0.00000000 {x:.2f} {y:.2f}\n")
        file.write("\n")

        file.write("# Mobile nodes\n")
        mote_index = len(fixed_positions)
        max_steps = 0
        mobile_trajectories = []

        for mote in mobile_motes:
            path_segments = mote["functionPath"]
            speed = mote["speed"]
            time_step = mote["timeStep"]
            is_round_trip = mote.get("isRoundTrip", False)
            
            print("path_segments", path_segments)

            # Avaliação dos segmentos
            x_all, y_all, segment_distances = [], [], []
            for x_expr, y_expr in path_segments:
                t_values = np.linspace(0, 1, num=100)
                x_vals = evaluate_function(x_expr, t_values)
                y_vals = evaluate_function(y_expr, t_values)
                x_all.append(x_vals)
                y_all.append(y_vals)
                segment_distances.append(np.sum(np.sqrt(np.diff(x_vals)**2 + np.diff(y_vals)**2)))

            total_distance = np.sum(segment_distances)
            total_duration = total_distance / speed
            total_steps = max(1, int(total_duration / time_step))
            max_steps = max(max_steps, total_steps)

            # Interpolação proporcional por segmento
            x_full, y_full = [], []
            for x_vals, y_vals, seg_dist in zip(x_all, y_all, segment_distances):
                proportion = seg_dist / total_distance if total_distance > 0 else 1
                seg_steps = max(1, int(proportion * total_steps))
                interp_t = np.linspace(0, 1, seg_steps)
                x_interp = np.interp(interp_t, np.linspace(0, 1, len(x_vals)), x_vals)
                y_interp = np.interp(interp_t, np.linspace(0, 1, len(y_vals)), y_vals)
                x_full.extend(x_interp)
                y_full.extend(y_interp)

            x_full = np.array(x_full)
            y_full = np.array(y_full)

            if is_round_trip:
                x_full = np.concatenate((x_full, x_full[::-1]))
                y_full = np.concatenate((y_full, y_full[::-1]))

            mobile_start_positions.append((x_full[0], y_full[0]))
            mobile_trajectories.append((mote_index, x_full, y_full, time_step))
            mote_index += 1

        for step in range(2 * max_steps):
            for mote_id, x_full, y_full, time_step in mobile_trajectories:
                if step < len(x_full):
                    file.write(f"{mote_id} {step * time_step:.8f} {x_full[step]:.2f} {y_full[step]:.2f}\n")
            file.write("\n")

    return fixed_positions, mobile_start_positions


### update_simulation_xml

A função a seguir realiza a geração de arquivos *xml* de configurações de simulação Cooja. O método utiliza um template e completa os dados com os parâmetros indicados.

In [6]:
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

def update_simulation_xml(
    fixed_positions: list[tuple[float, float]],
    mobile_positions: list[tuple[float, float]],
    root_motes: list[int],
    simulation_time: float,
    tx_range: float,
    interference_range: float,
    input_file: str,
    output_file: str
) -> None:
    """Atualiza arquivo XML de simulação com novos parâmetros.
    
    Args:
        fixed_positions: Lista de tuplas (x, y) com posições fixas
        mobile_positions: Lista de tuplas (x, y) com posições iniciais dos móveis
        root_motes: Lista de IDs dos motes servidores
        simulation_time: Tempo de simulação em minutos
        tx_range: Alcance de transmissão
        interference_range: Alcance de interferência
        inputFile: Caminho do arquivo XML de entrada (template)
        outputFile: Caminho do arquivo XML de saída
    """
    tree = ET.parse(input_file)
    root = tree.getroot()
    
    # Updates radio parameters
    radiomedium = root.find(".//radiomedium")
    if radiomedium is not None:
        transmitting_range = radiomedium.find("transmitting_range")
        if transmitting_range is not None:
            transmitting_range.text = str(tx_range)
        interference_range_elem = radiomedium.find("interference_range")
        if interference_range_elem is not None:
            interference_range_elem.text = str(interference_range)
    
    # Update simulation time in JS script keeping CDATA
    script_element = root.find(".//script")
    if script_element is not None and script_element.text is not None:
        script_text = script_element.text
        new_timeout = simulation_time * 60000  # Convertendo minutos para milissegundos
        script_text = script_text.replace("const timeOut = X * 1000;", f"const timeOut = {new_timeout} * 1000;")
        script_text = script_text.replace("TIMEOUT(X);", f"TIMEOUT({new_timeout + 1000});")
        script_element.text = f"<![CDATA[\n{script_text}\n]]>"
    
    # update motes
    motetype_root = root.find(".//motetype[description='server']")
    motetype_client = root.find(".//motetype[description='client']")
    
    if motetype_root is not None:
        for mote in motetype_root.findall("mote"):
            motetype_root.remove(mote)
    if motetype_client is not None:
        for mote in motetype_client.findall("mote"):
            motetype_client.remove(mote)
    
    for i, (x, y) in enumerate(fixed_positions + mobile_positions):
        mote_type = motetype_root if i + 1 in root_motes else motetype_client
        if mote_type is not None:
            mote = ET.SubElement(mote_type, "mote")
            
            interface_config = ET.SubElement(mote, "interface_config")
            interface_config.text = "org.contikios.cooja.interfaces.Position"
            ET.SubElement(interface_config, "pos", x=str(x), y=str(y))
            
            id_config = ET.SubElement(mote, "interface_config")
            id_config.text = "org.contikios.cooja.contikimote.interfaces.ContikiMoteID"
            ET.SubElement(id_config, "id").text = str(i + 1)
    
    xml_str = ET.tostring(root, encoding='utf-8')
    parsed_xml = minidom.parseString(xml_str)
    with open(output_file, "w", encoding="utf-8") as f:
        output = parsed_xml.toprettyxml(indent="  ")
        output = output.replace("?>", "encoding=\"UTF-8\"?>")
        output = output.replace("&gt;", ">")
        output = output.replace("&lt;", "<")
        output = output.replace("&quot;", "\"")
        output = output.replace("<![CDATA[\n", "<![CDATA[")
        output = output.replace("\n]]>", "]]>")
        
        # Remove blank lines, exceto dentro de CDATA
        inside_cdata = False
        lines_without_blanks = []
        for line in output.splitlines():
            if "<![CDATA[" in line:
                inside_cdata = True
            if inside_cdata or line.strip():
                lines_without_blanks.append(line)
            if "]]>" in line:
                inside_cdata = False
                
        final_content = "\n".join(lines_without_blanks)
        f.write(final_content)
    
    print(f"File {output_file} generated successfully!")

### convert_simulation_files

A função a seguir utiliza as funções de conversão e geração de arquivos de configurações para gerar os dois arquivos de configurações de simulações Cooja.

In [7]:

def convert_simulation_files(
    config: SimulationConfig, 
    template_file: str = "simulation_template.xml",
    outsim: str = "./tmp/simulation.xml",
    outpos: str = "./tmp/positions.dat"
    ):
    """Processa a simulação completa a partir dos arquivos de configuração."""    
    # Gera arquivo de posições e obtém posições iniciais
    fixed_positions, mobile_start_positions = generate_positions_from_json(
        config, 
        output_filename=outpos
    )
    
    # Identifica motes servidores (assume que o primeiro fixo é o servidor)
    root_motes = [1]
    
    # Gera arquivo XML de simulação
    update_simulation_xml(
        fixed_positions=fixed_positions,
        mobile_positions=mobile_start_positions,
        root_motes=root_motes,
        simulation_time=config["duration"],
        tx_range=50.0,
        interference_range=100.0,
        input_file=template_file,
        output_file=outsim
    )

### ssl/scp

Implementações Python para realização de conexão SSH e troca de arquivos com SCP.

In [8]:
import paramiko
from scp import SCPClient

def create_ssh_client(hostname, port, username, password):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(hostname, port=port, username=username, password=password)
    return client

def send_files_scp(client, local_path, remote_path, source_files, target_files):
    if len(source_files) != len(target_files):
        print("The number of source files is not equal number of targets.")
        return
    with SCPClient(client.get_transport()) as scp:
        for src, dest in zip(source_files, target_files):
            local_file_path = local_path + "/" + src
            remote_file_path = remote_path + "/" + dest
            print(f"Sending {local_file_path} to {remote_file_path}")
            scp.put(local_file_path, remote_file_path)  

### prepare_simulation_files

Função para recuperar arquivos do MongoDB no momento da execução das simulações.

In [9]:
import os

# Prepara arquivos para simulação, transferindo do MongoDB para sistema local
def prepare_simulation_files(sim: dict)-> tuple[list[str],list[str]]:
    local_files = []
    remote_files = []

    sim_id = str(sim["_id"])

    # Verifica se existe diretório temporário para manipulação de arquivos
    if os.path.exists("tmp") == False:
        os.mkdir("tmp")
        
    # Arquivos temporários da simulação
    local_xml = f"tmp/simulation_{sim_id}.xml"
    local_dat = f"tmp/positions_{sim_id}.dat"
        
    # Obtém um instância para uso do MongoDB
    mongo = mongo_exp_mgr_factory()

    # Baixar arquivos do GridFS
    mongo.save_file_from_mongo(sim["simulationFile"], local_xml)
    mongo.save_file_from_mongo(sim["positionsFile"], local_dat)
    
    local_files.extend([local_xml, local_dat])
    remote_files.extend(["simulation.csc", "positions.dat"])

    # Carrega arquivos extras do experimento
    exp = mongo.get_document_by_id("experiments", sim["experiment_id"])
    if exp and "linkedFiles" in exp:
        for f in exp["linkedFiles"]:
            fid = f["fileId"]
            local_path = f"tmp/{f['name']}" 
            mongo.save_file_from_mongo(fid, local_path)
            local_files.append(local_path)
            remote_name = f['name']
            if remote_name == 'simulation.xml':
                remote_name = 'simulation.csc'
            remote_files.append(remote_name)
    else:
        print(f"[WARN] Nenhum arquivo extra vinculado encontrado para o experimento {sim['experiment_id']}")

    return local_files, remote_files


## First Experiment

Gera uma coleção de simulações para o experimento.

In [10]:
def population_gen():
    if os.path.exists("tmp") == False:
        os.mkdir("tmp")
        
    mongo = mongo_exp_mgr_factory()
    
    exp = mongo.get_first_waiting_experiment()
    
    if exp != None:
        pop_size = exp["evolutiveParameters"]["populationSize"]
        gen_number = 1
        generation = {
            "experimentId": exp["_id"],
            "number": gen_number,
            "population": []
        }
        population = []
        sim_queue = []
        for n in range(pop_size):
            sim_model = exp["simulationModel"]
            amount = len(sim_model["simulationElements"]["fixedMotes"])
            positions = network_random_points(amount, sim_model["region"], sim_model["radiusOfReach"])
            for i in range(amount):
                if sim_model["simulationElements"]["fixedMotes"][i]["name"] != "server":
                    sim_model["simulationElements"]["fixedMotes"][i]["position"] = positions[i]
            
            sim_xml = f"./tmp/sim_{n}.xml"
            pos_dat = f"./tmp/pos_{n}.dat"
            
            convert_simulation_files(sim_model, outsim=sim_xml, outpos=pos_dat)
                        
            simfile_id = mongo.insert_file(sim_xml, f"gen_{gen_number}_sim_{n}")
            posfile_id = mongo.insert_file(pos_dat, f"gen_{gen_number}_pos_{n}")
            
            ind = {
                "simulationElements": sim_model,
                "simulationFile": simfile_id,
                "positionsFile": posfile_id,
                "simLogFile": None
            }
            population.append(ind)
            
            elem = {
                "experiment_id": exp["_id"],
                "generation_id": None,
                "simulationFile": simfile_id,
                "positionsFile": posfile_id,
                "status": "waiting"
            }
            sim_queue.append(elem)
            
        generation["population"] = population
        gen_id = mongo.insert_document(generation, "generations")
        
        for elem in sim_queue:
            elem["generation_id"] = gen_id
            mongo.insert_document(elem, "simqueue")
        
        exp["generations"] = [ gen_id ]
        mongo.update_experiment(exp["_id"], exp)
        
population_gen()

path_segments [['25 * np.cos(2 * np.pi * t)', '25 * np.sin(2 * np.pi * t)']]
path_segments [['100 * t - 50', '50'], ['50', '50 - 30 * t'], ['50 - 100 * t', '20'], ['-50', '20 + 30 * t']]
path_segments [['-150 + 60 * t', '-100'], ['-90', '-100 + 40 * t'], ['-90 - 60 * t', '-60'], ['-150', '-60 - 40 * t']]
path_segments [['90 + 80 * t', '120'], ['170', '120 - 50 * t'], ['170 - 80 * t', '70'], ['90', '70 + 50 * t']]
path_segments [['-100 * t + 50', '-50']]
path_segments [['100 * t - 100', '100 * t - 100']]
File ./tmp/sim_0.xml generated successfully!
path_segments [['25 * np.cos(2 * np.pi * t)', '25 * np.sin(2 * np.pi * t)']]
path_segments [['100 * t - 50', '50'], ['50', '50 - 30 * t'], ['50 - 100 * t', '20'], ['-50', '20 + 30 * t']]
path_segments [['-150 + 60 * t', '-100'], ['-90', '-100 + 40 * t'], ['-90 - 60 * t', '-60'], ['-150', '-60 - 40 * t']]
path_segments [['90 + 80 * t', '120'], ['170', '120 - 50 * t'], ['170 - 80 * t', '70'], ['90', '70 + 50 * t']]
path_segments [['-100 * t + 5

### Run Simulations

In [ ]:
import threading
import queue
from pathlib import Path

# Diretórios e credenciais SSH
LOCAL_DIRECTORY = '.'
REMOTE_DIRECTORY = '/opt/contiki-ng/tools/cooja'
LOCAL_LOG_DIR = "logs"
Path(LOCAL_LOG_DIR).mkdir(exist_ok=True)

CONTAINER_USERNAME = 'root'
CONTAINER_PASSWORD = 'root'
CONTAINER_HOSTNAME = 'localhost'
CONTAINER_PORTS = [2231, 2232, 2233, 2234, 2235]

def run_cooja_simulation(sim: dict, port, mongo: MongoExperimentManager):
    ssh = create_ssh_client(CONTAINER_HOSTNAME, port, CONTAINER_USERNAME, CONTAINER_PASSWORD)
    try:        
        sim_id = str(sim["_id"])
        
        print(f"[{port}] Iniciando simulação {sim_id}...")
        mongo.update_simulation_status(sim_id, "running")

        command = f"""
        cd ../{REMOTE_DIRECTORY} && \
        /opt/java/openjdk/bin/java --enable-preview -Xms4g -Xmx4g -jar build/libs/cooja.jar --no-gui simulation.csc
        """
        stdin, stdout, stderr = ssh.exec_command(command)

        for line in iter(stdout.readline, ""):
            print(f"[{port}][stdout] {line}", end="")
        for line in iter(stderr.readline, ""):
            print(f"[{port}][stderr] {line}", end="")

        # Copia o log da simulação para o sistema de arquivos local
        log_path = f"{LOCAL_LOG_DIR}/sim_{sim_id}.log"
        with SCPClient(ssh.get_transport()) as scp_client:
            print(f"[{port}] Copiando log da simulação para {log_path}")
            scp_client.get(f"{REMOTE_DIRECTORY}/COOJA.testlog", log_path)

        # Insere o log no GridFS e obtém o seu ID
        log_result_id = mongo.insert_file(log_path, "sim_result.log")
        print(f"[{port}] Simulação {sim_id} finalizada com sucesso. Log inserido com ID: {log_result_id}")

        mongo.simulation_done(sim, log_result_id)
        
    except Exception as e:
        print(f"[{port}] ERRO durante a simulação {sim_id}: {e}")
        mongo.update_simulation_status(sim_id, "error")
    finally:
        ssh.close()

def simulation_worker(sim_queue, port):
    mongo = mongo_exp_mgr_factory()
    while True:
        sim = sim_queue.get()
        if sim is None:
            break
        sim_id = str(sim["_id"])

        local_files, remote_files = prepare_simulation_files(sim)
        try:
            print(f"[{port}] Preparando simulação {sim_id}")
            ssh = create_ssh_client(CONTAINER_HOSTNAME, port, CONTAINER_USERNAME, CONTAINER_PASSWORD)
            send_files_scp(ssh, LOCAL_DIRECTORY, REMOTE_DIRECTORY, local_files, remote_files)
            ssh.close()

            run_cooja_simulation(sim, port, mongo)
            
        except Exception as e:
            print(f"[{port}] ERRO geral na simulação {sim_id}: {e}")
            mongo.update_simulation_status(sim_id, "error")
        finally:
            sim_queue.task_done()

def start_workers(num_workers=5):
    q = queue.Queue()
    for i in range(num_workers):
        t = threading.Thread(target=simulation_worker, args=(q, CONTAINER_PORTS[i]), daemon=True)
        t.start()
    print("[Sistema] Workers iniciados.")
    return q

def load_initial_waiting_jobs(sim_queue):
    print("[load] Buscando simulações pendentes no início...")
    mongo = mongo_exp_mgr_factory()
    pending_jobs = mongo.find_pending_simulations()
    count = 0
    for doc in pending_jobs:
        print(f"[load] Simulação pendente encontrada: {doc['_id']}")
        sim_queue.put(doc)
        count += 1
    print(f"[load] Total de simulações adicionadas à fila: {count}")


sim_queue = start_workers()
load_initial_waiting_jobs(sim_queue)


[Sistema] Workers iniciados.
[load] Buscando simulações pendentes no início...
[load] Simulação pendente encontrada: 68052f284b7b23d6ec07a56f
[load] Simulação pendente encontrada: 68052f284b7b23d6ec07a571
[load] Simulação pendente encontrada: 68052f294b7b23d6ec07a573
[load] Total de simulações adicionadas à fila: 3


[2233] Preparando simulação 68052f294b7b23d6ec07a573
[2232] Preparando simulação 68052f284b7b23d6ec07a571
[2231] Preparando simulação 68052f284b7b23d6ec07a56f
Sending ./tmp/simulation_68052f294b7b23d6ec07a573.xml to /opt/contiki-ng/tools/cooja/simulation.csc
Sending ./tmp/simulation_68052f284b7b23d6ec07a56f.xml to /opt/contiki-ng/tools/cooja/simulation.csc
Sending ./tmp/simulation_68052f284b7b23d6ec07a571.xml to /opt/contiki-ng/tools/cooja/simulation.csc
Sending ./tmp/positions_68052f284b7b23d6ec07a56f.dat to /opt/contiki-ng/tools/cooja/positions.datSending ./tmp/positions_68052f284b7b23d6ec07a571.dat to /opt/contiki-ng/tools/cooja/positions.dat

Sending ./tmp/positions_68052f294b7b23d6ec07a573.dat to /opt/contiki-ng/tools/cooja/positions.dat
Sending ./tmp/Makefile to /opt/contiki-ng/tools/cooja/Makefile
Sending ./tmp/Makefile to /opt/contiki-ng/tools/cooja/Makefile
Sending ./tmp/Makefile to /opt/contiki-ng/tools/cooja/Makefile
Sending ./tmp/project-conf.h to /opt/contiki-ng/tools/cooj